spotrac 迴歸分析

In [240]:
import sys
sys.path.append('G:\我的雲端硬碟\python')
import numpy as np
from polib.CsvEngn import *
from MLB_functions import *
from tqdm import tqdm

import statsmodels.api as sm #OLS
# https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html

from linearmodels.panel import PanelOLS # FE
# https://bashtage.github.io/linearmodels/doc/panel/models.html

# 讀檔

In [241]:
#野手
df_bat = pd.read_excel(r"G:\我的雲端硬碟\python\output\0904\Spotrac_bat_data_20210904_21.22.13.xlsx",
                        engine='openpyxl')
# 投手
df_pitch= pd.read_excel(r"G:\我的雲端硬碟\python\output\0904\Spotrac_pitch_data_20210904_21.22.17.xlsx",
                      engine='openpyxl')

# 特徵轉換

In [242]:
# # is_2020+2021
df_bat.loc[:,"is_gt_2020"] = (df_bat.loc[:,"Season"]>=2020)
df_pitch.loc[:,"is_gt_2020"] = (df_pitch.loc[:,"Season"]>=2020)

In [243]:
# for df in [df_pitch, df_bat]:
for df in [df_pitch,df_bat]:
    
    #T,F transform
    for col in [col for col in df_bat.columns if "is_" in col]:
        df.loc[:,col] = df.loc[:,col].apply(lambda v: 1 if v else 0)
        
    # AAV_dollar 取 自然對數
    df.loc[:,"AAV_dollar"] = np.log(df.loc[:,"AAV_dollar"])
    df.loc[np.isinf(df_bat.loc[:,"AAV_dollar"]),"AAV_dollar"] = np.nan
    
# #     NewCBA 2017~2021 dummy
#     df.loc[:,"NewCBA"] =df.loc[:,"Season"].apply(lambda s: 1 if (int(s)>=2017) & (int(s)<=2021) else 0)
    

# 年度篩選

In [244]:
# 篩選
df_bat = df_bat[(df_bat.loc[:,"Season"]<=2021) & (df_bat.loc[:,"Season"]>=2012)]
print(len(df_bat))
# df_bat = df_bat[(df_bat.loc[:,"Season"]<=2016) & (df_bat.loc[:,"Season"]>=2012)] #前五年
# df_bat = df_bat[(df_bat.loc[:,"Season"]>=2017) & (df_bat.loc[:,"Season"]<=2021)] #後五年
print(len(df_bat))

2567
2567


In [245]:
df_bat.Season.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
      dtype=int64)

In [83]:
# 篩選
df_pitch = df_pitch[(df_pitch.loc[:,"Season"]<=2021) & (df_pitch.loc[:,"Season"]>=2012)]
print(len(df_pitch))
# df_pitch = df_pitch[(df_pitch.loc[:,"Season"]<=2016) & (df_pitch.loc[:,"Season"]>=2012)]
# df_pitch = df_pitch[(df_pitch.loc[:,"Season"]>=2017) & (df_pitch.loc[:,"Season"]<=2021)]
print(len(df_pitch))

2550
2550


In [84]:
df_pitch.Season.unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
      dtype=int64)

In [44]:
# df_pitch.info()

In [45]:
# df_bat.info()

# Model 1

 WAR = 合約剩餘年數 + 一年合約dummy + 控制變數

## OLS

In [255]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手

In [256]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
#  "NewCBA",
    
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

3410
941


In [257]:
print(len(y),len(y))

941 941


In [258]:
# y.isna().sum()

In [259]:
# X.isna().sum()

In [260]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     8.410
Date:                Sat, 04 Sep 2021   Prob (F-statistic):           1.99e-38
Time:                        22:56:40   Log-Likelihood:                -1567.2
No. Observations:                 941   AIC:                             3212.
Df Residuals:                     902   BIC:                             3401.
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.0727      1.190     -0.902      0.367      -3.408       1.262
is_changeTeam        -0.4452      0.109     -4.089      0.000      -0.659      -0.232
Age                   0.0032      0.016      0.196      0.845      -0.029       0.035
Age^2             -1.621e-06   8.01e-06     -0.202      0.840   -1.73e-05    1.41e-05
YRS_left              0.1974      0.048      4.106      0.000       0.103       0.292
AAV_dollar            0.0946      0.071      1.324      0.186      -0.046       0.235
is_1year_contract    -0.0050      0.128     -0.039      0.969      -0.256       0.246
is_END_year          -0.9530      0.141     -6.771      0.000      -1.229      -0.677
Team_ATL              0.4375      0.354      1.235      0.217      -0.258       1.133
Team_BAL             -0.2382      0.461     -0.517      0.605      -1.143       0.666
Team_BOS              0.6026      0.338      1.781      0.075      -0.061       1.267
Team_CHC              0.0858      0.326      0.264      0.792      -0.553       0.725
Team_CHW              0.6092      0.350      1.742      0.082      -0.077       1.295
Team_CIN             -0.1026      0.350     -0.293      0.769      -0.789       0.584
Team_CLE              0.5366      0.375      1.431      0.153      -0.199       1.272
Team_COL             -0.3262      0.372     -0.878      0.380      -1.055       0.403
Team_DET             -0.1812      0.359     -0.505      0.614      -0.886       0.523
Team_HOU              0.4129      0.339      1.216      0.224      -0.253       1.079
Team_KC              -0.0420      0.342     -0.123      0.902      -0.713       0.629
Team_LAA             -0.3177      0.362     -0.878      0.380      -1.028       0.393
Team_LAD             -0.0070      0.330     -0.021      0.983      -0.655       0.641
Team_MIA             -0.2216      0.376     -0.590      0.556      -0.959       0.516
Team_MIL             -0.0390      0.342     -0.114      0.909      -0.711       0.633
Team_MIN             -0.1205      0.342     -0.353      0.724      -0.791       0.550
Team_NYM             -0.1897      0.345     -0.550      0.582      -0.866       0.487
Team_NYY              0.2798      0.357      0.784      0.433      -0.420       0.980
Team_OAK              0.6216      0.346      1.796      0.073      -0.058       1.301
Team_PHI              0.4934      0.347      1.421      0.156      -0.188       1.175
Team_PIT              0.4607      0.439      1.050      0.294      -0.400       1.321
Team_SD               0.0422      0.356      0.118      0.906      -0.657       0.741
Team_SEA              0.3276      0.361      0.908      0.364      -0.381       1.036
Team_SF              -0.1064      0.333     -0.319      0.749      -0.760       0.548
Team_STL             -0.1362      0.364     -0.374      0.709      -0.851

##  FE

In [237]:
df_pic = df_pitch.copy(deep=True) # 投手
# df_pic = df_bat.copy(deep=True) # 野手

In [238]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
#      "NewCBA"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

3410
1014


In [239]:
# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1543: RuntimeWarning: invalid value encountered in add
  x_arr = x_arr + x_gm


LinAlgError: SVD did not converge in Linear Least Squares

# Model 2

- WAR = (複數年)合約剩兩年       + 一年合約dummy + 控制變數
    - (複數年)合約剩三年
    - (複數年)合約剩四年以上
    - 基準: (複數年)合約剩一年

## OLS

In [52]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [53]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
 "Player_ID"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

2567
970


In [54]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X.astype(float)).fit()
est.summary()

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     10.02
Date:                Sat, 04 Sep 2021   Prob (F-statistic):           2.55e-56
Time:                        21:38:18   Log-Likelihood:                -1775.5
No. Observations:                 970   AIC:                             3649.
Df Residuals:                     921   BIC:                             3888.
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.5112      1.345     -0.380      0.704      -3.151       2.128
is_changeTeam        -0.8147      0.137     -5.932      0.000      -1.084      -0.545
Age                  -0.1266      0.020     -6.490      0.000      -0.165      -0.088
Age^2              6.167e-05   9.52e-06      6.477      0.000     4.3e-05    8.04e-05
AAV_dollar            0.4428      0.076      5.799      0.000       0.293       0.593
is_1year_contract     0.0832      0.173      0.480      0.631      -0.257       0.423
is_END_year          -1.3934      0.166     -8.372      0.000      -1.720      -1.067
is_left_2years       -0.2368      0.171     -1.384      0.167      -0.573       0.099
is_left_3years       -0.1314      0.216     -0.608      0.543      -0.555       0.293
is_left_ge4years      0.0735      0.203      0.362      0.717      -0.325       0.472
Team_ATL             -0.8005      0.351     -2.280      0.023      -1.490      -0.111
Team_BAL             -1.0352      0.425     -2.438      0.015      -1.868      -0.202
Team_BOS             -0.5210      0.345     -1.511      0.131      -1.197       0.155
Team_CHC             -0.6054      0.376     -1.609      0.108      -1.344       0.133
Team_CHW             -0.5792      0.394     -1.469      0.142      -1.353       0.195
Team_CIN             -0.3799      0.379     -1.002      0.317      -1.124       0.365
Team_CLE             -0.7475      0.350     -2.134      0.033      -1.435      -0.060
Team_COL             -1.5326      0.401     -3.819      0.000      -2.320      -0.745
Team_DET             -0.7742      0.398     -1.947      0.052      -1.555       0.006
Team_HOU              0.3890      0.384      1.014      0.311      -0.364       1.142
Team_KC              -0.6416      0.403     -1.591      0.112      -1.433       0.150
Team_LAA             -0.3657      0.352     -1.040      0.299      -1.056       0.324
Team_LAD              0.5094      0.395      1.289      0.198      -0.266       1.285
Team_MIA             -0.6373      0.386     -1.651      0.099      -1.395       0.120
Team_MIL              0.1100      0.369      0.298      0.766      -0.614       0.834
Team_MIN             -0.6329      0.423     -1.495      0.135      -1.464       0.198
Team_NYM             -0.4600      0.367     -1.253      0.211      -1.181       0.261
Team_NYY             -0.2346      0.357     -0.657      0.511      -0.935       0.466
Team_OAK             -0.3047      0.414     -0.737      0.461      -1.116       0.507
Team_PHI             -0.4524      0.403     -1.122      0.262      -1.244       0.339
Team_PIT             -0.5161      0.431     -1.196      0.232      -1.363       0.331
Team_SD              -0.7499      0.403     -1.862      0.063      -1.540       0.041
Team_SEA              0.3105      0.404      0.768      0.443      -0.483

##  FE

In [201]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [202]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

934
512


In [203]:
# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X.astype(float), 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3812
Estimator:                   PanelOLS   R-squared (Between):          -6.198e+04
No. Observations:                 512   R-squared (Within):               0.3812
Date:                Sat, Sep 04 2021   R-squared (Overall):          -2.873e+04
Time:                        22:39:52   Log-likelihood                   -677.10
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3.9485
Entities:                         223   P-value                           0.0000
Avg Obs:                       2.2960   Distribution:                  F(39,250)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             9.3174
                            

#  Model 3

WAR = 複數年合約的第一年  + 一年合約dummy + 控制變數

## OLS

In [183]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [184]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
     "NewCBA",
#     "is_gt_2020"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].replace([np.inf, -np.inf], np.nan).dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

1633
458


In [185]:
# X[["is_FIRST_year_BigContract","Age","Age^2"]]

In [186]:
# df_pic[["WAR_SHIRK","is_FIRST_year_BigContract"]].corr()

In [187]:
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')

# plt.scatter(df_pic['WAR_SHIRK'],df_pic['is_FIRST_year_BigContract'],color='blue')
# plt.show()

In [188]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     5.593
Date:                Sat, 04 Sep 2021   Prob (F-statistic):           1.72e-22
Time:                        22:36:55   Log-Likelihood:                -853.71
No. Observations:                 458   AIC:                             1799.
Df Residuals:                     412   BIC:                             1989.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.0494      2.087     -0.982      0.327      -6.151       2.052
is_changeTeam                -1.0026      0.203     -4.948      0.000      -1.401      -0.604
Age                          -0.1377      0.030     -4.586      0.000      -0.197      -0.079
Age^2                      6.715e-05   1.47e-05      4.578      0.000    3.83e-05     9.6e-05
AAV_dollar                    0.5752      0.113      5.103      0.000       0.354       0.797
is_1year_contract             0.3118      0.267      1.168      0.243      -0.213       0.836
is_FIRST_year_BigContract     0.3234      0.208      1.554      0.121      -0.086       0.733
is_END_year                  -1.3276      0.259     -5.133      0.000      -1.836      -0.819
Team_ATL                     -1.3745      0.589     -2.335      0.020      -2.532      -0.217
Team_BAL                     -0.2980      0.727     -0.410      0.682      -1.726       1.130
Team_BOS                     -0.5090      0.567     -0.898      0.370      -1.623       0.605
Team_CHC                     -0.7881      0.603     -1.308      0.192      -1.973       0.397
Team_CHW                     -0.9641      0.622     -1.550      0.122      -2.186       0.258
Team_CIN                     -0.7749      0.579     -1.338      0.182      -1.913       0.363
Team_CLE                     -1.2856      0.584     -2.200      0.028      -2.434      -0.137
Team_COL                     -1.2940      0.672     -1.924      0.055      -2.616       0.028
Team_DET                     -0.6300      0.628     -1.003      0.317      -1.865       0.605
Team_HOU                      0.0125      0.716      0.017      0.986      -1.394       1.419
Team_KC                      -1.3982      0.721     -1.940      0.053      -2.815       0.019
Team_LAA                     -0.1601      0.609     -0.263      0.793      -1.358       1.038
Team_LAD                      0.4034      0.703      0.574      0.566      -0.978       1.785
Team_MIA                     -1.0161      0.587     -1.731      0.084      -2.170       0.138
Team_MIL                     -0.4170      0.601     -0.694      0.488      -1.599       0.765
Team_MIN                     -1.1767      0.681     -1.727      0.085      -2.516       0.162
Team_NYM                     -0.0634      0.645     -0.098      0.922      -1.331       1.204
Team_NYY                     -1.0099      0.561     -1.800      0.073      -2.113       0.093
Team_OAK                     -1.1829      0.680     -1.740      0.083      -2.519       0.153
Team_PHI                     -0.4227      0.753     -0.561      0.575      -1.904       1.058
Team_PIT                     -0.5983      0.637     -0.939      0.348      -1.851       0.654
Team_SD                      -1.1427      0.656     -1.742      0.082      

##  FE

In [207]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [208]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID",
     "NewCBA"
#     "is_gt_2020"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR_SHIRK"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR_SHIRK"]+ID_cols].copy(deep=True)

934
512


In [209]:
# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X, 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:              WAR_SHIRK   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -230.78
No. Observations:                 512   R-squared (Within):               0.1806
Date:                Sat, Sep 04 2021   R-squared (Overall):             -83.308
Time:                        22:41:03   Log-likelihood                   -812.65
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.5012
Entities:                         223   P-value                           0.0382
Avg Obs:                       2.2960   Distribution:                  F(37,252)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             10.619
                            

# Model 4

## OLS

In [86]:
# df_bat[df_bat["Player"].isin(["Mookie Betts"])][["Season","Player","WAR","WAR_SHIRK","NewCBA"]]

In [87]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [88]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
 'WAR_lag_1_MA3',
 'WAR_SHIRK',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID",
    "is_gt_2020",
#      "NewCBA"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR_SHIRK"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR_SHIRK"].copy(deep=True)

2567
968


In [89]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              WAR_SHIRK   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     2.919
Date:                Sat, 04 Sep 2021   Prob (F-statistic):           1.52e-09
Time:                        21:56:48   Log-Likelihood:                -1796.3
No. Observations:                 968   AIC:                             3685.
Df Residuals:                     922   BIC:                             3909.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -9.0730      1.369     -6.629      0.000     -11.759      -6.387
is_changeTeam         0.3052      0.127      2.399      0.017       0.056       0.555
Age                   0.0641      0.019      3.341      0.001       0.026       0.102
Age^2             -3.123e-05   9.36e-06     -3.336      0.001   -4.96e-05   -1.29e-05
AAV_dollar            0.4593      0.075      6.099      0.000       0.312       0.607
is_1year_contract     0.1637      0.158      1.037      0.300      -0.146       0.473
is_END_year           0.7970      0.162      4.917      0.000       0.479       1.115
Team_ATL              0.0997      0.360      0.277      0.782      -0.606       0.806
Team_BAL              0.1023      0.435      0.235      0.814      -0.751       0.955
Team_BOS              0.2528      0.353      0.715      0.475      -0.441       0.946
Team_CHC              0.2410      0.385      0.626      0.532      -0.515       0.997
Team_CHW              0.4568      0.405      1.129      0.259      -0.337       1.251
Team_CIN              0.2942      0.388      0.758      0.449      -0.468       1.056
Team_CLE              0.4728      0.359      1.317      0.188      -0.232       1.178
Team_COL              0.2719      0.410      0.662      0.508      -0.534       1.078
Team_DET              0.5491      0.407      1.350      0.177      -0.249       1.348
Team_HOU             -0.2413      0.401     -0.601      0.548      -1.028       0.546
Team_KC               0.3097      0.413      0.751      0.453      -0.500       1.119
Team_LAA              0.8804      0.359      2.451      0.014       0.175       1.585
Team_LAD              0.1868      0.404      0.462      0.644      -0.607       0.980
Team_MIA              0.1110      0.396      0.281      0.779      -0.665       0.887
Team_MIL              0.3494      0.378      0.925      0.355      -0.392       1.091
Team_MIN              0.3898      0.434      0.899      0.369      -0.461       1.241
Team_NYM              0.2241      0.375      0.597      0.551      -0.513       0.961
Team_NYY              0.0076      0.366      0.021      0.983      -0.711       0.726
Team_OAK             -0.2487      0.422     -0.589      0.556      -1.077       0.580
Team_PHI             -0.1303      0.412     -0.316      0.752      -0.940       0.679
Team_PIT             -0.1591      0.442     -0.360      0.719      -1.026       0.707
Team_SD               0.2067      0.412      0.502      0.616      -0.601       1.015
Team_SEA             -0.0864      0.414     -0.208      0.835      -0.900       0.727
Team_SF               0.1797      0.394      0.456      0.648      -0.593       0.953
Team_STL              0.1976      0.376      0.525      0.600      -0.541       0.936
Team_TB               0.4685      0.430      1.091      0.276      -0.375